<a href="https://colab.research.google.com/github/valid999/NLP-TensorFlow_Projects/blob/main/NLP%26Bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake New Classifier using LSTM

* id: unique id for a news article
* title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable

In [193]:
import numpy as np
import pandas as pd

In [194]:
df = pd.read_csv('train.csv' , engine = 'python' ,  encoding='utf-8'  , on_bad_lines='skip')

In [195]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [196]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [197]:
df.shape

(20822, 5)

In [198]:
df = df.dropna()

In [199]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [200]:
df.shape

(18285, 5)

In [201]:
df = df.dropna()

In [202]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [203]:
df.drop([19782 , 19781] , axis = 0 , inplace = True)


In [204]:
## Get the independent features
X = df.drop('label' , axis = 1)

In [205]:
y = df['label']

In [206]:
df['label'].unique()

array(['1', '0'], dtype=object)

In [207]:
(df.loc[(df['label'] ==' чтобы это была дорога с двусторонним движением.'  )])

,id,title,author,text,label


In [208]:
## Check the whether datraset is balanced or not
y.value_counts()

0    10361
1     7922
Name: label, dtype: int64

In [209]:
import tensorflow as tf

In [210]:
tf.__version__

'2.14.0'

In [211]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [212]:
## Vocabulary size
voc_size = 5000

In [213]:
messages = X.copy()

In [214]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [215]:
import nltk
import re
from nltk.corpus import stopwords

In [216]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [217]:
messages.reset_index(inplace = True)

In [218]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0 , len(messages)):
  print(i)
  review = re.sub('[^a-zA-A]' , ' ' , messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)


Streaming output truncated to the last 5000 lines.
13283
13284
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
1

In [219]:
corpus[-1]

'hat eep aliv'

In [220]:
from nltk.corpus import stopwords

In [221]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [222]:
onhot_repr = [one_hot(words , voc_size) for words in corpus]
onhot_repr

[[3345, 4810, 2715, 1121, 147, 2764, 2564, 4910, 4530, 2772, 4713, 3005, 3792],
 [227, 2651, 31, 4422, 3434, 4444],
 [2066, 4376, 2261, 4726, 1550, 3837],
 [115, 1860, 1988, 3529, 4020, 1337, 2777, 1395],
 [4891, 1557, 423, 4994, 3127, 2222, 1557, 2106, 989, 833],
 [4151,
  4713,
  3164,
  1579,
  1425,
  2453,
  1121,
  225,
  1099,
  228,
  3597,
  2827,
  2525,
  277,
  3469,
  4444],
 [1049, 3005, 28, 1320, 2004, 1457, 4560, 1342, 430, 2067, 492],
 [3597, 2465, 4670, 694, 870, 544, 2453, 2053, 430, 2067, 492],
 [196, 3185, 1322, 2359, 1617, 323, 3451, 1207, 2453, 344],
 [1132, 1980, 282, 1457, 3898, 3629],
 [3448, 1565, 4266, 1292, 3478, 1885, 3929, 1623, 4443, 3875, 2694],
 [551, 2564, 323, 3686, 2453, 870],
 [1121, 2819, 2715, 267, 4184, 1988, 3057, 1254, 3139, 1283, 3868],
 [1988, 3773, 2579, 3307, 1831, 4105, 3358, 1379, 430, 2067, 492],
 [291, 3116, 4360, 2811, 375, 430, 2067, 492],
 [942, 4515, 4782, 3682, 3736, 4900, 154, 4038, 1142, 3207],
 [1272, 1988, 1988, 227],
 [4814, 

## Embedding Representation

In [223]:
sent_length = 20
embedded_docs = pad_sequences(onhot_repr , padding = 'pre' , maxlen = sent_length)

embedded_docs[-1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 1691, 2023, 3460], dtype=int32)

In [241]:
from keras.src.optimizers import optimizer
from sklearn.utils import optimize
import keras
## Creating modl
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size , embedding_vector_features , input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
## Train test split
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_6 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [242]:
import numpy as np
X_final = np.array(embedded_docs , dtype = 'f')
y_final = np.array(y , dtype = 'f')

In [243]:
## Train test split

from sklearn.model_selection import train_test_split
X_train , X_test , y_train  , y_test  = train_test_split(X_final , y_final , test_size = 0.25 , random_state = 42)

In [248]:
## Model training
model.fit(X_train , y_train , validation_data =  (X_test , y_test) , epochs = 30 , batch_size = 32 )

Epoch 1/30
429/429 [==============================] - 16s 26ms/step - loss: 0.2432 - accuracy: 0.8917 - val_loss: 0.1869 - val_accuracy: 0.9188
Epoch 2/30
429/429 [==============================] - 4s 10ms/step - loss: 0.1286 - accuracy: 0.9510 - val_loss: 0.2126 - val_accuracy: 0.9206
Epoch 3/30
429/429 [==============================] - 5s 11ms/step - loss: 0.0930 - accuracy: 0.9660 - val_loss: 0.2227 - val_accuracy: 0.9217
Epoch 4/30
429/429 [==============================] - 7s 17ms/step - loss: 0.0641 - accuracy: 0.9779 - val_loss: 0.2682 - val_accuracy: 0.9217
Epoch 5/30
429/429 [==============================] - 4s 8ms/step - loss: 0.0432 - accuracy: 0.9865 - val_loss: 0.3234 - val_accuracy: 0.9136
Epoch 6/30
429/429 [==============================] - 4s 8ms/step - loss: 0.0267 - accuracy: 0.9918 - val_loss: 0.3492 - val_accuracy: 0.9107
Epoch 7/30
429/429 [==============================] - 4s 10ms/step - loss: 0.0194 - accuracy: 0.9938 - val_loss: 0.4517 - val_accuracy: 0.9099


In [249]:
y_pred = model.predict(X_test)


143/143 [==============================] - 1s 3ms/step


In [250]:
import numpy as np

y_pred = np.where(y_pred>= 0.5 , 1 , 0)

In [251]:
from sklearn.metrics import confusion_matrix , accuracy_score
confusion_matrix(y_test , y_pred)

array([[2372,  243],
       [ 158, 1798]])

In [252]:
print(accuracy_score(y_test , y_pred))

0.9122730255961496
